Get research output data from Zoteron and prepare the data for FileMaker import. 

This procedure is typically run 2 weeks after the 
finalisation of mid-year and annual KPI reporting.

In [2]:
import json
import sqlite3
import csv
import pandas as pd
import importlib
from lib import kpi

In [13]:
# Run this if changes have been made to kpi.py. It ensure that any changes are
# made available in the current session.
import importlib
importlib.reload(kpi)

<module 'kpi' from '/Users/nxo/Workspace/saef_api/kpi.py'>

In [27]:
# This is an ALL load to fix historical error. Hopefully it's a one off
with open('data/all/saef_library_All_postman_20240110.json', mode='r', encoding='utf8') as f:
        data = f.read()
        saef_library = json.loads(data)
projectKey = []
# This is output from an FM report: ProjectKey_Report
with open('data/all/fm_projectKey.csv', newline='') as f:
        reader = csv.reader(f)
        for row in reader:
                projectKey.append(row)

data           = kpi.load_data()
rpt_config     = kpi.get_rpt_args()
responses_json = kpi.split_response(data[0])
ppl_collection = kpi.person_construct(responses_json, data[2], rpt_config[0])
proj_saef      = kpi.project_construct(responses_json, ppl_collection[0])
buckets        = kpi.matched_library(saef_library, ppl_collection[1])
meta_bucket    = buckets[0]
bucket         = buckets[1]
ppl_saef       = ppl_collection[0]
ppl_hash       = ppl_collection[1]


In [28]:
meta_bucket.loc[meta_bucket['itemType'] == 'book', 'itemType']              = 'Book'
meta_bucket.loc[meta_bucket['itemType'] == 'dataset', 'itemType']           = 'Dataset'
meta_bucket.loc[meta_bucket['itemType'] == 'journalArticle', 'itemType']    = 'Journal Article'
meta_bucket.loc[meta_bucket['itemType'] == 'presentation', 'itemType']      = 'Presentation'
meta_bucket.loc[meta_bucket['itemType'] == 'plenary', 'itemType']           = 'Plenary'
meta_bucket.loc[meta_bucket['itemType'] == 'report', 'itemType']            = 'Report'
meta_bucket.loc[meta_bucket['itemType'] == 'radioBroadcast', 'itemType']    = 'Radio'
meta_bucket.loc[meta_bucket['itemType'] == 'artwork', 'itemType']           = 'Artwork'
meta_bucket.loc[meta_bucket['itemType'] == 'tvBroadcast', 'itemType']       = 'TV'
meta_bucket.loc[meta_bucket['itemType'] == 'bookSection', 'itemType']       = 'Book Chapter'
meta_bucket.loc[meta_bucket['itemType'] == 'conferencePaper', 'itemType']   = 'Conference Paper'
meta_bucket.loc[meta_bucket['itemType'] == 'newspaperArticle', 'itemType']  = 'Newspaper'
meta_bucket.loc[meta_bucket['itemType'] == 'videoRecording', 'itemType']    = 'Video'
meta_bucket.loc[meta_bucket['itemType'] == 'blogPost', 'itemType']          = 'Blog'
meta_bucket.loc[meta_bucket['itemType'] == 'magazineArticle', 'itemType']   = 'Magazine'
meta_bucket.loc[meta_bucket['itemType'] == 'film', 'itemType']              = 'Film'

In [37]:
con = sqlite3.connect("data/all/saef_library.db")
cur = con.cursor()

In [31]:
# We should not need to re-create these tables !
# cur.execute("CREATE TABLE fm_zotero(id_zotero, item_type, title, rights, pubyr, name, id_person, pub_title, project)")
# cur.execute("CREATE TABLE fm_projectKey(id_project, project_code, project_alias)")
# cur.executemany("INSERT INTO fm_projectKey VALUES(?, ?, ?)", projectKey)
# con.commit()

In [32]:
# Match bucket headings with fm_zotero table headings. Then append to fm_zotero
meta_bucket.rename(columns={"key": "id_zotero", "itemType": "item_type", "publicationTitle": "pub_title"}, errors="raise", inplace=True)
meta_bucket.to_sql('fm_zotero', con, if_exists='append', index=False)

471

In [18]:
# Update the following tables in FM
# Outputs
# people_Outputs
# project_Outputs

In [38]:
# Export FM ready(ish) project_Ouputs table csv ingest
# outputs = pd.read_sql_query("SELECT DISTINCT id_zotero, item_type, title, pub_title, LOWER(rights), pubyr, project, id_project \
#                             FROM fm_zotero LEFT JOIN fm_projectKey \
#                             ON fm_zotero.project = fm_projectKey.project_code \
#                             WHERE project != ''", con)
# outputs.to_csv('data/output/proj_outputs_20240110.csv',index=False)
# Visual check, run before export
# for row in cur.execute("SELECT DISTINCT id_zotero, item_type, title, pub_title, LOWER(rights), pubyr, project, id_project \
#                         FROM fm_zotero LEFT JOIN fm_projectKey \
#                         ON fm_zotero.project = fm_projectKey.project_code \
#                        WHERE project != ''"):
#     print(row)

In [34]:
# Export FM ready(ish) Ouputs table csv ingest
ppl_outputs = pd.read_sql_query("SELECT id_person, id_zotero\
                                FROM fm_zotero", con)
ppl_outputs.to_csv('data/output/ppl_outputs_20240110.csv',index=False)
# Visual check, run before export
# for row in cur.execute("SELECT id_person, id_project, title, pub_title, pubyr \
#                         FROM fm_zotero LEFT JOIN fm_projectKey \
#                         ON fm_zotero.project = fm_projectKey.project_code \
#                        WHERE project != ''"):
#     print(row)

In [120]:
# Future buckets should perform an update into fm_zotero of new rows only
# ToDo: write update SQL logic
# Outputs list
# .header on
# .mode csv
# .once saef_api/data/outputs_20231005.csv
# SELECT id_zotero, item_type, title, LOWER(rights) AS embargo, pubyr, pub_title FROM fm_zotero;
# .once saef_api/data/ppl_outputs_20231005.csv
# SELECT id_person, id_zotero FROM fm_zotero;
# .once saef_api/data/projects_Outputs.csv
# SELECT DISTINCT project, id_project, id_zotero from fm_zotero LEFT JOIN fm_projectKey ON fm_zotero.project = fm_projectKey.project_code WHERE project != '';

# Tidy data in Excel, save as .xlsx, FM import

In [35]:
con.close()

In [46]:
for row in cur.execute("SELECT DISTINCT id_zotero, item_type, title, pub_title, LOWER(rights), pubyr, project FROM fm_zotero", con):
    print(row)

ValueError: parameters are of unsupported type